In [19]:
#@title Mount drive
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/CEvolutiva/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/CEvolutiva/data


In [20]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
#@title Imports
import operator
import math
import random
import numpy
import pandas as pd
import itertools
from deap import algorithms, base, creator, tools, gp
import csv
import random

In [22]:
#@title Converage
with open('ca.txt') as f:
  lines = f.readlines()
size = int(lines[0])
lines = lines[1:]
table = []
for line in lines:
  pcx, pm, iter = line.split(' ')[:-1]
  table.append((int(pcx), int(pm), int(iter)))

pcx = numpy.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
pm = numpy.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
iter = numpy.array([10, 30, 50, 70, 100, 120])

In [23]:
filename = "mux.csv"

problem_examples = open(filename)
reader = csv.reader(problem_examples)
raw = [row for row in reader]
names = raw[0]
problem = [list(map(lambda x: x == '1', instance)) for instance in raw[1:]]

In [24]:
len(problem)

64

In [25]:
# Define a new primitive set for strongly typed GP.
pset = gp.PrimitiveSetTyped("MAIN", itertools.repeat(bool, 6), bool, "IN")

# Boolean operators.
pset.addPrimitive(operator.and_, [bool, bool], bool)
pset.addPrimitive(operator.or_, [bool, bool], bool)
pset.addPrimitive(operator.not_, [bool], bool)

# Logic operators.
# Define a new if-then-else function
def if_then_else(input, output1, output2):
    if input: return output1
    else: return output2

pset.addPrimitive(if_then_else, [bool, bool, bool], bool)

pset.addTerminal(False, bool)
pset.addTerminal(True, bool)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=2, max_=4)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalProblem(individual):
    # Transform the tree expression in a callable function.
    func = toolbox.compile(expr=individual)
    # Evaluate the sum of correctly classified examples.
    # In this case 64 will be the maximum score for the 6-input multiplexer.
    real = 64
    prediction = sum(bool(func(*ins[:6])) is ins[6] for ins in problem)
    error = math.sqrt(real**2 - prediction**2) / len(problem)
    return error,

toolbox.register("evaluate", evalProblem)
toolbox.register("select", tools.selDoubleTournament, fitness_size=3, parsimony_size=1.1, fitness_first=True)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=2, max_=4)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))

In [26]:
solutions = []
def run():
  for CXPB, MUTPB, NGEN in table[:25]:
      pop = toolbox.population(n=500)
      hof = tools.HallOfFame(1)
      stats = tools.Statistics(lambda ind: ind.fitness.values)
      stats.register("avg", numpy.mean)
      stats.register("std", numpy.std)
      stats.register("min", numpy.min)
      stats.register("max", numpy.max)
      
      pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=pcx[CXPB], mutpb=pm[MUTPB], ngen=iter[NGEN], stats=stats, halloffame=hof)
      solutions.append((hof[0], (pcx[CXPB], pm[MUTPB], iter[NGEN])))

if __name__ == "__main__":
    random.seed(42)
    run()

gen	nevals	avg     	std      	min     	max     
0  	500   	0.853769	0.0522318	0.624218	0.949918
1  	101   	0.80809 	0.0543471	0.484123	0.927025
2  	97    	0.773242	0.0505079	0.484123	0.949918
3  	89    	0.737959	0.0573838	0.484123	0.927025
4  	89    	0.696745	0.0565778	0.484123	0.927025
5  	85    	0.663949	0.0496052	0.484123	0.866025
6  	97    	0.644726	0.0531531	0.484123	0.899218
7  	81    	0.62257 	0.0553997	0.484123	0.866025
8  	98    	0.60022 	0.0765007	0.484123	0.927025
9  	104   	0.558582	0.0827631	0.484123	0.866025
10 	94    	0.515996	0.0704324	0.484123	0.866025
11 	91    	0.509949	0.0726925	0.484123	0.899218
12 	98    	0.509925	0.0707172	0.484123	0.899218
13 	89    	0.505242	0.0623582	0.484123	0.927025
14 	107   	0.509189	0.070377 	0.484123	0.949918
15 	98    	0.504616	0.0649297	0.484123	0.927025
16 	99    	0.508148	0.0717625	0.484123	0.927025
17 	90    	0.503867	0.0623018	0.484123	0.899218
18 	71    	0.500982	0.0587589	0.484123	0.883331
19 	69    	0.500084	0.0542599	0.484123	0

In [27]:
best, params = solutions[0]
#print(f"{best.fitness.values[0]} {best} {params}")
it = 1
while it < len(solutions):
  solution, sparams = solutions[it]
  #print(f"{solution.fitness.values[0]} {solution} {sparams}")
  if best.fitness.values[0] > solution.fitness.values[0]:
    best, params = solution, sparams
  it += 1

print(best)
print(best.fitness.values[0])
print(params)

if_then_else(IN1, if_then_else(True, if_then_else(and_(IN1, IN0), IN5, and_(IN3, IN3)), IN3), or_(and_(and_(IN1, IN3), and_(IN5, IN4)), or_(if_then_else(IN0, IN4, IN2), and_(IN1, IN0))))
0.0
(0.1, 0.1, 120)


In [29]:
func = toolbox.compile(expr=best)
sum(bool(func(*ins[:6])) is ins[6] for ins in problem)

64